In [1]:
import pymongo
import pandas as pd
import json
import joblib
from sklearn.ensemble import IsolationForest

# --- CONFIGURAÇÕES ---
MONGO_CONNECTION_STRING = ""
MONGO_DB_NAME = "iotdb"
MONGO_COLLECTION_LEITURAS = "leituras_estufa"

# Definimos novos nomes de ficheiros para não substituir os antigos
ARQUIVO_MODELO_V2 = 'modelo_anomalia_v2.pkl'
ARQUIVO_ESTATISTICAS_V2 = 'estatisticas_normalidade_v2.json'

print("--- Iniciando Treino Final com Dataset Completo ---")

--- Iniciando Treino Final com Dataset Completo ---


In [2]:
# --- 1. Carregar os Dados (Sem Limite) ---
try:
    client = pymongo.MongoClient(MONGO_CONNECTION_STRING)
    db = client[MONGO_DB_NAME]
    collection = db[MONGO_COLLECTION_LEITURAS]
    
    # Carrega TODOS os dados, sem o .limit()
    dados_historicos = list(collection.find())
    df = pd.DataFrame(dados_historicos)
    
    print(f"Total de {len(df)} registos carregados do banco de dados.")
except Exception as e:
    print(f"Erro ao carregar dados do MongoDB: {e}")

# --- 2. Preparar os Dados ---
df['timestamp_full'] = pd.to_datetime(df['timestamp_full'])
df['hora_do_dia'] = df['timestamp_full'].dt.hour
features = ['hora_do_dia', 'temperatura', 'umidade_ar', 'umidade_solo', 'luminosidade_lux']
df_treinamento = df[features].dropna()

print(f"Dados preparados. {len(df_treinamento)} registos válidos para treino.")

Total de 94191 registos carregados do banco de dados.
Dados preparados. 94191 registos válidos para treino.


In [3]:
# --- 3. Treinar o Novo Modelo de Anomalia ---
print("Iniciando o treinamento do modelo de Detecção de Anomalias (Isolation Forest)...")
# (Isto pode demorar alguns minutos com 94.000 registos)
model = IsolationForest(contamination='auto', random_state=42, n_jobs=-1) # n_jobs=-1 usa todos os cores do CPU
model.fit(df_treinamento)
print("Modelo treinado com sucesso!")

Iniciando o treinamento do modelo de Detecção de Anomalias (Isolation Forest)...
Modelo treinado com sucesso!


In [4]:
# --- 4. Salvar o Novo Modelo ---
joblib.dump(model, ARQUIVO_MODELO_V2)
print(f"Novo modelo salvo com sucesso no ficheiro: '{ARQUIVO_MODELO_V2}'")

Novo modelo salvo com sucesso no ficheiro: 'modelo_anomalia_v2.pkl'


In [5]:
# --- 5. Calcular as Novas Estatísticas de Normalidade ---
print("\nCalculando Estatísticas Horárias (Média e Desvio Padrão) com o dataset completo...")
estatisticas = df_treinamento.groupby('hora_do_dia').agg(['mean', 'std']).to_dict()

estatisticas_formatadas = {}
for hora in range(24):
    estatisticas_formatadas[str(hora)] = {
        'temperatura': {
            'media': estatisticas.get(('temperatura', 'mean'), {}).get(hora, 0),
            'desvio_padrao': estatisticas.get(('temperatura', 'std'), {}).get(hora, 0)
        },
        'umidade_ar': {
            'media': estatisticas.get(('umidade_ar', 'mean'), {}).get(hora, 0),
            'desvio_padrao': estatisticas.get(('umidade_ar', 'std'), {}).get(hora, 0)
        },
        'umidade_solo': {
            'media': estatisticas.get(('umidade_solo', 'mean'), {}).get(hora, 0),
            'desvio_padrao': estatisticas.get(('umidade_solo', 'std'), {}).get(hora, 0)
        },
        'luminosidade_lux': {
            'media': estatisticas.get(('luminosidade_lux', 'mean'), {}).get(hora, 0),
            'desvio_padrao': estatisticas.get(('luminosidade_lux', 'std'), {}).get(hora, 0)
        }
    }
print("Novas estatísticas calculadas.")


Calculando Estatísticas Horárias (Média e Desvio Padrão) com o dataset completo...
Novas estatísticas calculadas.


In [6]:
# --- 6. Salvar as Novas Estatísticas ---
with open(ARQUIVO_ESTATISTICAS_V2, 'w') as f:
    json.dump(estatisticas_formatadas, f, indent=4)
print(f"Novas estatísticas salvas com sucesso no ficheiro: '{ARQUIVO_ESTATISTICAS_V2}'")

print("\n--- PROCESSO DE RE-TREINO CONCLUÍDO ---")

Novas estatísticas salvas com sucesso no ficheiro: 'estatisticas_normalidade_v2.json'

--- PROCESSO DE RE-TREINO CONCLUÍDO ---
